## Modification (DWoo): Loading data files directly from Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
base_dir_path = '/gdrive/MyDrive/Forecasting - DSI Capstone Spring \'21/Colabs'

# 1. Main setup

In [ ]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [ ]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [ ]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [ ]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
START_TRAIN = 1069         # Last day in train set
END_TRAIN = 1798       # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

Sample training date range: 2014-01-01 (d_1069) to 2015-12-31 (d_1798)

In [ ]:
########################### Load Data
#################################################################################
print('Load Main Data')

# Here are reafing all our data 
# without any limitations and dtype modification
train_df = pd.read_csv(os.path.join(base_dir_path, 'data/sales_train_validation_ca1.csv'))
prices_df = pd.read_csv(os.path.join(base_dir_path, 'data/sell_prices_ca1.csv'))
calendar_df = pd.read_csv(os.path.join(base_dir_path, 'data/calendar.csv'))

Load Main Data


In [ ]:
[print(df.shape) for df in [train_df, prices_df, calendar_df]]

(3049, 736)
(321293, 4)
(1969, 14)


[None, None, None]

In [ ]:
# # Reduce dataset by just looking at one store
# train_df = train_df.loc[train_df.store_id == 'CA_1']

In [ ]:
def make_grid():
  ########################### Make Grid
  #################################################################################
  print('Create Grid')

  # Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
  # and labels are 'd_' coulmns

  index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
  grid_df = pd.melt(train_df, 
                    id_vars = index_columns, 
                    var_name = 'd', 
                    value_name = TARGET)

  print('Train rows:', len(train_df), len(grid_df))

  # To be able to make predictions
  # we need to add "test set" to our grid
  add_grid = pd.DataFrame()
  for i in range(1,29):
      temp_df = train_df[index_columns]
      temp_df = temp_df.drop_duplicates()
      temp_df['d'] = 'd_'+ str(END_TRAIN+i)
      temp_df[TARGET] = np.nan
      add_grid = pd.concat([add_grid,temp_df])

  grid_df = pd.concat([grid_df,add_grid])
  grid_df = grid_df.reset_index(drop=True)

  # # Let's check our memory usage
  print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

  # We can free some memory 
  # by converting "strings" to categorical
  # it will not affect merging and 
  # we will not lose any valuable data
  for col in index_columns:
      grid_df[col] = grid_df[col].astype('category')

  # Let's check again memory usage
  print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

  return grid_df

### Store-product level features

In [ ]:
def item_release_date(item_grid_df):
  ########################### Product Release date
  #################################################################################
  print('Release week')

  # It seems that leadings zero values
  # in each train_df item row
  # are not real 0 sales but mean
  # absence for the item in the store
  # we can safe some memory by removing
  # such zeros

  # Prices are set by week
  # so it we will have not very accurate release week 
  release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
  release_df.columns = ['store_id','item_id','release']

  # Now we can merge release_df
  item_grid_df = merge_by_concat(item_grid_df, release_df, ['store_id','item_id'])
  del release_df

  # We want to remove some "zeros" rows
  # from item_grid_df 
  # to do it we need wm_yr_wk column
  # let's merge partly calendar_df to have it
  item_grid_df = merge_by_concat(item_grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                        
  # Now we can cutoff some rows 
  # and safe memory 
  item_grid_df = item_grid_df[item_grid_df['wm_yr_wk']>=item_grid_df['release']]
  item_grid_df = item_grid_df.reset_index(drop=True)

  # Let's check our memory usage
  print("{:>20}: {:>8}".format('Original item_grid_df',sizeof_fmt(item_grid_df.memory_usage(index=True).sum())))

  # Should we keep release week 
  # as one of the features?
  # Only good CV can give the answer.
  # Let's minify the release values.
  # Min transformation will not help here 
  # as int16 -> Integer (-32768 to 32767)
  # and our item_grid_df['release'].max() serves for int16
  # but we have have an idea how to transform 
  # other columns in case we will need it
  item_grid_df['release'] = item_grid_df['release'] - item_grid_df['release'].min()
  item_grid_df['release'] = item_grid_df['release'].astype(np.int16)

  # Let's check again memory usage
  print("{:>20}: {:>8}".format('Reduced item_grid_df',sizeof_fmt(item_grid_df.memory_usage(index=True).sum())))  

  return item_grid_df

def make_item_price_features():
  ########################### Prices
  #################################################################################
  print('Prices')

  price_feature_df = prices_df.copy()

  # We can do some basic aggregations
  price_feature_df['price_max'] = price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
  price_feature_df['price_min'] = price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
  price_feature_df['price_std'] = price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
  price_feature_df['price_mean'] = price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

  # and do price normalization (min/max scaling)
  price_feature_df['price_norm'] = price_feature_df['sell_price']/price_feature_df['price_max']

  # Some items are can be inflation dependent
  # and some items are very "stable"

  price_feature_df['price_nunique'] = price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique') 
  price_feature_df['item_nunique'] = price_feature_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

  # I would like some "rolling" aggregations
  # but would like months and years as "window"
  calendar_prices = calendar_df[['wm_yr_wk','month','year']]
  calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk']) # distinct(.keep_all = True)
  price_feature_df = price_feature_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
  del calendar_prices

  # Now we can add price "momentum" (some sort of)
  # Shifted by week 
  # by month mean
  # by year mean
  price_feature_df['price_momentum'] = price_feature_df['sell_price']/price_feature_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
  price_feature_df['price_momentum_m'] = price_feature_df['sell_price']/price_feature_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
  price_feature_df['price_momentum_y'] = price_feature_df['sell_price']/price_feature_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

  return price_feature_df.drop(['month', 'year'], axis=1)

In [ ]:
def make_calendar_features(grid_df):
  ########################### Merge calendar
  #################################################################################
  # grid_df = grid_df[MAIN_INDEX]

  # Merge calendar partly
  icols = ['date',
          'd',
          'event_name_1',
          'event_type_1',
          'event_name_2',
          'event_type_2',
          'snap_CA']

  grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

  # Minify data
  # 'snap_' columns we can convert to bool or int8
  icols = ['event_name_1',
          'event_type_1',
          'event_name_2',
          'event_type_2',
          'snap_CA']

  for col in icols:
      grid_df[col] = grid_df[col].astype('category')

  # Convert to DateTime
  grid_df['date'] = pd.to_datetime(grid_df['date'])

  # Make some features from date
  grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
  grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
  grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
  grid_df['tm_y'] = grid_df['date'].dt.year
  grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
  grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

  grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8) 
  grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

  # Convert 'd' to int
  grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

  return grid_df


In [ ]:
def make_lag_features(grid_df):
  # We need only 'id','d','sales'
  # to make lags and rollings
  lag_df = grid_df[['id','d','sales']]
  SHIFT_DAY = 28

  # Lags
  # with 28 day shift
  start_time = time.time()
  print('Create lags')

  LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
  lag_df = lag_df.assign(**{
          '{}_lag_{}'.format(col, l): lag_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
          for l in LAG_DAYS
          for col in [TARGET]
      })

  # Minify lag columns
  for col in list(lag_df):
      if 'lag' in col:
          lag_df[col] = lag_df[col].astype(np.float16)

  print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

  # Rollings
  # with 28 day shift
  start_time = time.time()
  print('Create rolling aggs')

  for i in [7,14,30,60,180]:
      print('Rolling period:', i)
      lag_df['rolling_mean_'+str(i)] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
      lag_df['rolling_std_'+str(i)]  = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

  # Rollings
  # with sliding shift
  for d_shift in [1,7,14]: 
      print('Shifting period:', d_shift)
      for d_window in [7,14,30,60]:
          col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
          lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
      
      
  print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

  return lag_df

def make_agg_lag_features(grid_df, groupby_col):
  # We need only groupby_col, 'd','sales'
  # to make lags and rollings
  lag_df = grid_df[[groupby_col, 'd','sales']]
  SHIFT_DAY = 28

  # Lags
  # with 28 day shift
  start_time = time.time()
  print('Create lags')

  LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
  lag_df = lag_df.assign(**{
          '{}_lag_{}'.format(col, l): lag_df.groupby([groupby_col])[col].transform(lambda x: x.shift(l))
          for l in LAG_DAYS
          for col in [TARGET]
      })

  # Minify lag columns
  for col in list(lag_df):
      if 'lag' in col:
          lag_df[col] = lag_df[col].astype(np.float16)

  print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

  # Rollings
  # with 28 day shift
  start_time = time.time()
  print('Create rolling aggs')

  for i in [7,14,30,60,180]:
      print('Rolling period:', i)
      lag_df['rolling_mean_'+str(i)] = lag_df.groupby([groupby_col])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
      lag_df['rolling_std_'+str(i)]  = lag_df.groupby([groupby_col])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

  # Rollings
  # with sliding shift
  for d_shift in [1,7,14]: 
      print('Shifting period:', d_shift)
      for d_window in [7,14,30,60]:
          col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
          lag_df[col_name] = lag_df.groupby([groupby_col])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
      
      
  print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

  return lag_df

In [ ]:
def make_mean_encoding_feature(grid_df, icols):
  ########################### Apply on grid_df
  #################################################################################
  # lets read grid from 
  # https://www.kaggle.com/kyakovlev/m5-simple-fe
  # to be sure that our grids are aligned by index
  
  # grid_df['sales'][grid_df['d']>(1941-28)] = np.nan

  for col in icols:
      print('Encoding', col)
      col_name = '_'+'_'.join(col)+'_'
      grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)['sales'].transform('mean').astype(np.float16)
      grid_df['enc'+col_name+'std'] = grid_df.groupby(col)['sales'].transform('std').astype(np.float16)

  return grid_df

### Product-store level data

In [ ]:
product_grid_df = make_grid()
product_grid_df = item_release_date(product_grid_df)
price_feature_df = make_item_price_features()

product_grid_df = reduce_mem_usage(product_grid_df)
price_feature_df = reduce_mem_usage(price_feature_df)

# Merge Prices
original_columns = list(product_grid_df)
product_grid_df = product_grid_df.merge(price_feature_df, on=['store_id','item_id','wm_yr_wk'], how='left')
product_grid_df = make_calendar_features(product_grid_df)

product_grid_df = reduce_mem_usage(product_grid_df)

Create Grid
Train rows: 3049 2225770
    Original grid_df: 141.1MiB
     Reduced grid_df:  53.1MiB
Release week
Original item_grid_df:  85.5MiB
Reduced item_grid_df:  72.7MiB
Prices
Mem. usage decreased to 47.11 Mb (35.2% reduction)
Mem. usage decreased to 14.40 Mb (60.8% reduction)
Mem. usage decreased to 166.39 Mb (0.0% reduction)


In [ ]:
# Create lag features
lag_df = make_lag_features(product_grid_df)
lag_cols = [col for col in lag_df.columns if col != 'sales']
product_grid_df = product_grid_df.merge(lag_df[lag_cols], how='left', on=['id', 'd'])

Create lags
2.33 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
4.05 min: Lags


In [ ]:
product_grid_icols = [
  ['store_id'],
  ['cat_id'],
  ['dept_id'],
  ['store_id', 'cat_id'],
  ['store_id', 'dept_id'],
  ['item_id'],
  ['item_id', 'store_id']
]
product_grid_df = make_mean_encoding_feature(product_grid_df, product_grid_icols)
product_grid_df = reduce_mem_usage(product_grid_df)

Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'store_id']
Mem. usage decreased to 383.85 Mb (0.0% reduction)


In [ ]:
# Store-product label encoding
dynamic_cat_cols = [col for col in product_grid_df.columns if 'event_' in col or 'tm_' in col]
obj_cols = [col for col in dynamic_cat_cols if 'tm_' not in col]
tmp_cols = [col for col in dynamic_cat_cols if 'tm_' in col]
for col in obj_cols:
  le = LabelEncoder()
  le.fit(product_grid_df[col].astype(str))
  product_grid_df[f'{col}_lencoded'] = le.transform(product_grid_df[col].astype(str))

dynamic_cat_cols = tmp_cols + [f'{col}_lencoded' for col in obj_cols]

In [ ]:
# Drop the original event columns
product_grid_df.drop(columns=obj_cols, inplace=True)

### Store-level data

In [ ]:
store_grid_df = make_grid()
store_grid_df = store_grid_df.groupby(['store_id', 'd']).agg(dict(sales=sum)).reset_index()
store_grid_df = make_calendar_features(store_grid_df)
store_grid_df = reduce_mem_usage(store_grid_df)

# restore prediction horizon back to NaN 
store_grid_df.loc[(store_grid_df.d>END_TRAIN),'sales'] = np.nan

Create Grid
Train rows: 3049 2225770
    Original grid_df: 141.1MiB
     Reduced grid_df:  53.1MiB
Mem. usage decreased to  0.03 Mb (14.4% reduction)


In [ ]:
# Create lag features
lag_df = make_agg_lag_features(store_grid_df, 'store_id')
lag_cols = [col for col in lag_df.columns if col != 'sales']
store_grid_df = store_grid_df.merge(lag_df[lag_cols], how='left', on=['store_id', 'd'])

Create lags
0.00 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
0.00 min: Lags


In [ ]:
store_grid_icols = [
  ['store_id'],
]
store_grid_df = make_mean_encoding_feature(store_grid_df, store_grid_icols)
store_grid_df = reduce_mem_usage(store_grid_df)

Encoding ['store_id']
Mem. usage decreased to  0.08 Mb (0.0% reduction)


In [ ]:
# Store label encoding
dynamic_cat_cols = [col for col in store_grid_df.columns if 'event_' in col or 'tm_' in col]
obj_cols = [col for col in dynamic_cat_cols if 'tm_' not in col]
tmp_cols = [col for col in dynamic_cat_cols if 'tm_' in col]
for col in obj_cols:
  le = LabelEncoder()
  le.fit(store_grid_df[col].astype(str))
  store_grid_df[f'{col}_lencoded'] = le.transform(store_grid_df[col].astype(str))

dynamic_cat_cols = tmp_cols + [f'{col}_lencoded' for col in obj_cols]

In [ ]:
# Drop the original event columns
store_grid_df.drop(columns=obj_cols, inplace=True)

### Save training data to pickle files

In [ ]:
product_grid_df.to_pickle(os.path.join(base_dir_path, 'preprocessed_data/train_ca1_store_product.pkl'))
store_grid_df.to_pickle(os.path.join(base_dir_path, 'preprocessed_data/train_ca1_store.pkl'))

### Aggregate test data to store level

In [ ]:
test_df = pd.read_csv(os.path.join(base_dir_path, 'data/sales_train_evaluation_ca1.csv'))

d_cols = [col for col in test_df.columns if 'd_' in col]


In [ ]:
store_test_df = test_df.groupby('store_id')[d_cols].sum().reset_index()

store_test_df = pd.melt(
    store_test_df,
    id_vars = 'store_id', 
    var_name = 'd',
    value_name = TARGET
).merge(
    calendar_df[['d', 'date']],
    how='left',
    on='d'
)
store_test_df = store_test_df.assign(
    d=store_test_df.d.apply(lambda x: int(x[2:]))
)

In [ ]:
store_test_df.to_pickle(os.path.join(base_dir_path, 'preprocessed_data/test_ca1_store_no_features.pkl'))

### Process test data for store-product level

In [ ]:
product_store_df = pd.melt(
    test_df[['item_id', 'store_id'] + d_cols],
    id_vars = ['item_id', 'store_id'],
    var_name = 'd',
    value_name = TARGET
).merge(
    calendar_df[['d', 'date']],
    how='left',
    on='d'
)

product_store_df = product_store_df.assign(
    d=product_store_df.d.apply(lambda x: int(x[2:]))
)

In [ ]:
product_store_df.to_pickle(os.path.join(base_dir_path, 'preprocessed_data/test_ca1_store_product_no_features.pkl'))